In [36]:
from pathlib import Path
import cv2
import numpy as np
from openvino.runtime import Core
import cv2, time, onnxruntime

# 载入ImageNet标签
session = onnxruntime.InferenceSession(f'bicycle_weights/best.onnx', providers=['CPUExecutionProvider'])
meta = session.get_modelmeta().custom_metadata_map  # metadata
imagenet_labels = eval(meta['names'])

# 初始化 OpenVINO Core
core = Core()

# 加载电动车和头盔检测模型
model_path = Path("bicycle_weights/best_openvino_model/best.xml")
compiled_model = core.compile_model(model=model_path, device_name="CPU")

model_path_helmet = Path("runs/detect/train7/weights/best_openvino_model/best.xml")
compiled_model_helmet = core.compile_model(model=model_path_helmet, device_name="CPU")

# 测试图片集
test_images = ["test_image_1.jpg", "test_image_2.png"]
final_results = []

# 确保所有输入图像尺寸一致，适配网络输入要求
input_node = compiled_model.inputs[0]    # yolov8n-cls只有一个输入节点
# N, C, H, W = input_node.shape # 获得输入张量的形状
H, W = 512, 512
output_node = compiled_model.outputs[0]  # yolov8n-cls只有一个输出节点
ir = compiled_model.create_infer_request()
##########################################
#   ---根据模型定义预处理和后处理函数-------
##########################################

# 定义预处理函数
def preprocess(image, new_shape=(W,H)):
    # Preprocess image data from OpenCV
    [height, width, _] = image.shape
    length = max((height, width))
    letter_box = np.zeros((length, length, 3), np.uint8)
    letter_box[0:height, 0:width] = image
    blob = cv2.dnn.blobFromImage(image, scalefactor=1 / 255, size=new_shape, swapRB=True)
    return blob

# 定义后处理函数
def postprocess(outs):
    score = np.max(outs)
    id = np.argmax(outs)
    return score, id, imagenet_labels[id]

##########################################
#   ----- AI同步推理计算 ------------
##########################################
# 采集图像
image = cv2.imread(test_images[1])
# 数据预处理
blob = preprocess(image)
# 执行推理计算并获得结果
outs = ir.infer(blob)[output_node]
# 对推理结果进行后处理
score, id, label = postprocess(outs)

##########################################
#   ----- 统计带前后预处理的AI推理性能------
##########################################
start = time.time()
N = 4000
for i in range(N):
    blob = preprocess(image)
    outs = ir.infer(blob)[output_node]
    score, id, label = postprocess(outs)
FPS = N / (time.time() - start) 

##########################################
#   ----- 后处理结果集成到AI应用程序 -------
##########################################
# 显示处理结果
msg = f"YOLOv5s-cls Result:{label}, Score:{score:4.2f}, FPS:{FPS:4.2f}"
cv2.putText(image, msg, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 100, 200), 2)
cv2.imshow("YOLOv5s-cls OpenVINO Sync Infer Demo",image)
cv2.waitKey()
cv2.destroyAllWindows()


KeyError: 21345

In [31]:
a = np.array(outs.data)
print(a.shape)
print(a[0][0])

(1, 10, 5376)
[     2.5893      24.829      29.298 ...      429.54      443.47      461.93]


In [28]:
from ultralytics import YOLO
model_ = YOLO(r"bicycle_weights\best.pt")
results = model_(test_images)
print(results[0].boxes)

0: 640x640 7 motorbikes, 19.9ms
1: 640x640 5 motorbikes, 19.9ms
Speed: 3.0ms preprocess, 19.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([4., 4., 4., 4., 4., 4., 4.], device='cuda:0')
conf: tensor([0.9084, 0.8214, 0.7787, 0.7605, 0.5886, 0.3811, 0.2893], device='cuda:0')
data: tensor([[4.2809e+02, 4.2317e+02, 5.5294e+02, 5.7350e+02, 9.0841e-01, 4.0000e+00],
        [6.6189e+02, 4.2971e+02, 7.8865e+02, 6.2152e+02, 8.2142e-01, 4.0000e+00],
        [1.0113e+03, 4.3394e+02, 1.0951e+03, 5.4577e+02, 7.7873e-01, 4.0000e+00],
        [0.0000e+00, 4.5578e+02, 1.9604e+02, 7.2000e+02, 7.6046e-01, 4.0000e+00],
        [1.1030e+03, 4.3458e+02, 1.1461e+03, 5.0438e+02, 5.8860e-01, 4.0000e+00],
        [9.2572e+02, 4.4165e+02, 9.7931e+02, 5.0921e+02, 3.8109e-01, 4.0000e+00],
        [7.6975e+02, 4.3267e+02, 8.7460e+02, 5.4604e+02, 2.8931e-01, 4.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: